## Practice Final Exam Questions

### Warm-up Questions

1.)

Print the number of elements in the RDD B

Print the first five elements of RDD B

In [1]:
n=10000;
B=sc.parallelize(range(n))

## Your answer here
print B.count()
print B.take(5)

10000
[0, 1, 2, 3, 4]


2.) 

Given an RDD of words, find and output one of the longest words

In [3]:
words=['this','is','the','best','mac','ever','jupyter']
wordRDD=sc.parallelize(words)

## Your answer here
wordRDD.map(lambda strI:(len(strI),strI)).sortBy(lambda x : -x[0]).take(1)

[(7, 'jupyter')]

3.)

Remove duplicate elements in RDD

In [2]:
DuplicateRDD = sc.parallelize([1,1,2,2,3,3])

## Your answer here
DuplicateRDD.distinct().collect()

[2, 1, 3]

4.)

Given an RDD, create a new RDD where each element appears twice

In [98]:
A=sc.parallelize(range(5))
A.union(A).collect()
A.flatMap(lambda x:[x,x]).collect()

[0, 0, 1, 1, 2, 2, 3, 3, 4, 4]

5.)

Count how many positive numbers are there in the RDD?

In [13]:
B=sc.parallelize(range(-10,20))

B.filter(lambda x:x>0).count()
## Your answer here

20

### Pair RDD Questions

6.)

Compute and print the largest value for each key in this pair RDD

In [1]:
PairRDD = sc.parallelize([(1,2), (2,4), (2,6)])

def joinList(a,b):
    return a.append(b)
## Your answer here
PairRDD.groupByKey().map(lambda (key,val):(key,max(val))).collect()

[(2, 6), (1, 2)]

In [9]:
PairRDD.groupByKey().map(lambda (y,x):(y,list(x))).collect()

[(2, [4, 6]), (1, [2])]

    7.)

Sort a pair RDD by key and print the result

In [28]:
PairRDD = sc.parallelize([(2,2),(1,4),(3,6),(2,1)])
PairRDD.sortBy(lambda x:x[0]).collect()
## Your answer here

[(1, 4), (2, 2), (2, 1), (3, 6)]

8.)

Perform the following transformation:

Original RDD : `[(1, 2), (2, 4), (2, 6)]`

After transformation :  `[(2, [4, 6]), (1, [2])]`

In [39]:
map(lambda (x,y) : (x,list(y)),PairRDD.groupByKey().collect())

[(2, [4, 6]), (1, [2])]

9.)

Given two pair RDDs A and B, create the following RDD

`[('adam', ('kalai', None)),
 ('vaclav', (None, 'M')),
 ('john', ('dow', 'M')),
 ('beth', ('simon', 'F'))]`

In [53]:
A=sc.parallelize([('john','dow'),('adam','kalai'),('beth','simon')])
B=sc.parallelize([('beth','F'),('john','M'),('vaclav','M')])

A.fullOuterJoin(B).collect()
## Your answer here


[('adam', ('kalai', None)),
 ('vaclav', (None, 'M')),
 ('john', ('dow', 'M')),
 ('beth', ('simon', 'F'))]

### Statistics Questions

10.)

Suppose X is an RDD where each element is a floating point value. Write code to **efficiently** compute a good **approximation** of the median value?

In [3]:
from numpy.random import rand
X=sc.parallelize(rand(100000)/2)
Y=X.sample(False,0.1)
n=Y.count()
print sorted(Y.collect())[int(n/2)]
Q=X.collect()

## Your answer here

0.249921223168


In [71]:
import matplotlib as plt

In [76]:
%pylab inline
histogram(Q)

Populating the interactive namespace from numpy and matplotlib


100000

11.)

For the same RDD in Q-10, compute the mean and the standard deviation.

In [9]:
X=sc.parallelize(rand(100000)/2)
Y=X.sample(False,0.1)
nX=X.count()
nY=Y.count()
sumX=X.reduce(lambda x,y : x+y)
sumY=Y.reduce(lambda x,y : x+y)
stdevX=X.stdev()
stdevY=Y.sampleStdev()
meanX=sumX/nX
meanY=sumY/nY
print meanX, meanY, stdevX, stdevY

0.25027964752 0.249227065048 0.143941802129 0.14413127852


12.) 

Suppose R is an RDD of tuples, each tuple containing two floating point numbers $(x,y)$. Compute the covariance of $x$ and $y$ using a single pass over the RDD.

In [38]:
import os
import sys

from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import StructField,StructType, FloatType

In [44]:
sql=SQLContext(sc)

In [73]:
from numpy.random import rand
n=10000
a=rand(n); b=rand(n)
R=sc.parallelize(zip(5*a+b,5*a-b))
R=R.map(lambda (x,y):(float(x),float(y)))
# two passes
w= R.map(lambda (x,y):(x,y,1)).reduce(lambda x,y : (x[0]+y[0],x[1]+y[1],x[2]+y[2])) # add x,y,counts
x_bar=w[0]/w[2] # mean of x
y_bar=w[1]/w[2] # mean of y

cov1=R.map(lambda (x,y):(x-x_bar)*(y-y_bar)/w[2]).reduce(lambda x,y:x+y) # covariance definition and sumation using reduce

In [74]:
df=sql.createDataFrame(R,schema)
cov2=df.stat.cov('x','y')
print cov1-cov2

-0.000198760776548


13.)

Suppose R is an RDD that contains integer numbers in the range 0 to 3. Write code to efficiently compute and plot an approximate histogram.

In [96]:
X=([0]*10000+[1]*23000+[2]*15532+[3]*10000)
keys=rand(len(X))
R=sc.parallelize(zip(keys,X)).cache()
R=R.repartitionAndSortWithinPartitions(2).map(lambda x:x[1])
R.histogram([0,1,2,3,4,5])
R.histogram(3)
## Your answer here

([0, 1, 2, 3], [10000, 23000, 25532])

histogram(self, buckets)
	source code 

Compute a histogram using the provided buckets. The buckets are all open to the right except for the last which is closed. e.g. [1,10,20,50] means the buckets are [1,10) [10,20) [20,50], which means 1<=x<10, 10<=x<20, 20<=x<=50. And on the input of 1 and 50 we would have a histogram of 1,0,1.

If your histogram is evenly spaced (e.g. [0, 10, 20, 30]), this can be switched from an O(log n) inseration to O(1) per element(where n = # buckets).

Buckets must be sorted and not contain any duplicates, must be at least two elements.

If `buckets` is a number, it will generates buckets which are evenly spaced between the minimum and maximum of the RDD. For example, if the min value is 0 and the max is 100, given buckets as 2, the resulting buckets will be [0,50) [50,100]. buckets must be at least 1 If the RDD contains infinity, NaN throws an exception If the elements in RDD do not vary (max == min) always returns a single bucket.

It will return an tuple of buckets and histogram.

>>> rdd = sc.parallelize(range(51))
>>> rdd.histogram(2)
([0, 25, 50], [25, 26])
>>> rdd.histogram([0, 5, 25, 50])
([0, 5, 25, 50], [5, 20, 26])
>>> rdd.histogram([0, 15, 30, 45, 60])  # evenly spaced buckets
([0, 15, 30, 45, 60], [15, 15, 15, 6])
>>> rdd = sc.parallelize(["ab", "ac", "b", "bd", "ef"])
>>> rdd.histogram(("a", "b", "c"))
(('a', 'b', 'c'), [2, 2])

In [89]:
X=([0]*10+[1]*23+[2]*15+[3]*10)
Y=([4]*15+[5]*18+[6]*5+[7]*20)
#keys=rand(len(X))
R=sc.parallelize(zip(X,Y)).cache()

In [90]:
R.repartitionAndSortWithinPartitions(8).glom().collect()

[[(0, 4),
  (0, 4),
  (0, 4),
  (0, 4),
  (0, 4),
  (0, 4),
  (0, 4),
  (0, 4),
  (0, 4),
  (0, 4)],
 [(1, 4),
  (1, 4),
  (1, 4),
  (1, 4),
  (1, 4),
  (1, 5),
  (1, 5),
  (1, 5),
  (1, 5),
  (1, 5),
  (1, 5),
  (1, 5),
  (1, 5),
  (1, 5),
  (1, 5),
  (1, 5),
  (1, 5),
  (1, 5),
  (1, 5),
  (1, 5),
  (1, 5),
  (1, 5),
  (1, 5)],
 [(2, 6),
  (2, 6),
  (2, 6),
  (2, 6),
  (2, 6),
  (2, 7),
  (2, 7),
  (2, 7),
  (2, 7),
  (2, 7),
  (2, 7),
  (2, 7),
  (2, 7),
  (2, 7),
  (2, 7)],
 [(3, 7),
  (3, 7),
  (3, 7),
  (3, 7),
  (3, 7),
  (3, 7),
  (3, 7),
  (3, 7),
  (3, 7),
  (3, 7)],
 [],
 [],
 [],
 []]